In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
import threading
import queue
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [6]:
# Function to read frames from the video capture
def read_frames(cap, frame_queue):
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_queue.put(frame)

In [7]:
# Function to process frames
def process_frames(frame_queue):
    while True:
        frame = frame_queue.get()
        # Perform your image processing here

In [3]:
cap = cv2.VideoCapture(0)
# Set up a queue to share frames between threads
frame_queue = queue.Queue(maxsize=5)
# used to record the time when we processed last frame 
prev_frame_time = 0
  
# used to record the time at which we processed current frame 
new_frame_time = 0

while cap.isOpened():
    ret, image = cap.read()
    # image = cv2.flip(image, 1)
    # # image = cv2.resize(image, VID_DIM)
    
    # # To improve performance, optionally mark the image as not writeable to
    # # pass by reference.
    # image.flags.writeable = False
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # # Draw the hand annotations on the image.
    # image.flags.writeable = True
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    new_frame_time = time.time() 
  
    # Calculating the fps 
  
    # fps will be number of frame processed in given time frame 
    # since their will be most of time error of 0.001 second 
    # we will be subtracting it to get more accurate result 
    fps = 1/(new_frame_time-prev_frame_time) 
    prev_frame_time = new_frame_time 

    cv2.putText(image, 'FPS: ' + str(int(fps)), (225,12), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.imshow("Mediapipe Feed", image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 



In [2]:
size_of_image = [640, 480]

def calculate_angle_for_left_curl(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    shoulder = [left_shoulder.x, left_shoulder.y]
    elbow = [left_elbow.x, left_elbow.y]
    wrist = [left_wrist.x, left_wrist.y]

    angle = calculate_angle(shoulder, elbow, wrist)

    position_of_text = np.multiply(elbow, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_right_curl(landmarks):
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

    shoulder = [right_shoulder.x, right_shoulder.y]
    elbow = [right_elbow.x, right_elbow.y]
    wrist = [right_wrist.x, right_wrist.y]

    angle = calculate_angle(shoulder, elbow, wrist)

    position_of_text = np.multiply(elbow, size_of_image).astype(int)

    return angle, position_of_text

In [4]:
import cv2
import time
import queue
import threading
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture(0)
frame_queue = queue.Queue(maxsize=5)
prev_frame_time = 0
new_frame_time = 0
counter = 0
stage = None
def read_frames():
    global frame_queue
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            image, results = process_image(image, holistic)
            frame_queue.put(image)


def process_image(image, holistic):
    image = cv2.flip(image, 1)
    # image = cv2.resize(image, VID_DIM)
    
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    return image, results


def process_frames():
    global frame_queue, prev_frame_time, counter, stage
    cc = cv2.VideoWriter_fourcc(*'XVID')
    file = cv2.VideoWriter('output.avi', cc, 15.0, (640, 480))

    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                break

            new_frame_time = time.time() 
        
            # Calculating the fps 
        
            # fps will be number of frame processed in given time frame 
            # since their will be most of time error of 0.001 second 
            # we will be subtracting it to get more accurate result 
            fps = 1/(new_frame_time-prev_frame_time) 
            prev_frame_time = new_frame_time 

            image, results = process_image(image, holistic)

            try:
                landmarks = results.pose_landmarks.landmark
                left_curl_angle, left_curl_position_of_text = calculate_angle_for_left_curl(landmarks)
                
                # # Visualize angle
                # cv2.putText(image, 
                #             str(left_curl_angle), 
                #             tuple(left_curl_position_of_text), 
                #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 
                #             2, 
                #             cv2.LINE_AA
                # )
                rigth_curl_angle, right_curl_position_of_text = calculate_angle_for_right_curl(landmarks)
                
                # Visualize angle
                # cv2.putText(image, 
                #             str(rigth_curl_angle), 
                #             tuple(right_curl_position_of_text), 
                #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 
                #             2, 
                #             cv2.LINE_AA
                # )
                # Curl counter logic
                if rigth_curl_angle > 160:
                    stage = "down"
                if rigth_curl_angle < 30 and stage =='down':
                    stage="up"
                    counter +=1
                    print(counter)
                            
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (325,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (115,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (100,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'FPS: ' + str(int(fps)), (225,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)


            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Hands', image)

            # Write the frame to the video file
            file.write(image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                    file.release()
                    break

width, height = 640, 480  # Set your desired width and height
read_thread = threading.Thread(target=read_frames)
process_thread = threading.Thread(target=process_frames)

read_thread.start()
process_thread.start()

read_thread.join()
process_thread.join()

cap.release()
cv2.destroyAllWindows()
